In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


def pegar_dataframe_por_num_resposta(num_resposta: int):
    return pd.read_csv('perguntas_2019 - {} respostas - Pronto para uso (com bias).csv'.format(num_resposta))#%%



In [2]:
for num_respostas in range(3, 6):
    print("\n\n - - - - - Começando teste com {} respostas - - - - - \n\n".format(num_respostas))
    df = pegar_dataframe_por_num_resposta(num_respostas)

    colunas_respostas = ['pontos_resposta', 'tempo_resposta', 'user_reputation', 'tem_site_proprio', 'maestria_tags']
    colunas_relevantes = [c + str(resposta) for resposta in range(0, num_respostas) for c in colunas_respostas]

    x = df[colunas_relevantes]
    y = df['alvo']

    simple_imputer = SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=0)
    simple_imputer.fit(x)

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15)

    X_train = simple_imputer.transform(X_train)
    X_test = simple_imputer.transform(X_test)

    # Para motivos de teste ignoramos a etapa de scaling
    # scaler = StandardScaler()
    # scaler.fit(X_train)
    # X_train = scaler.transform(X_train)
    # X_test = scaler.transform(X_test)

    # Cria modelo
    mlp = MLPClassifier(hidden_layer_sizes=(3, 3), max_iter=3000)
    mlp.fit(X_train, y_train.values.ravel())

    y_pred = mlp.predict(X_test)

    cm = confusion_matrix(y_test, y_pred)

    print(cm)
    print(classification_report(y_test, y_pred))

    print("\n\n - - - - - Terminado teste com {} respostas - - - - - \n\n".format(num_respostas))



 - - - - - Começando teste com 3 respostas - - - - - 


[[26747     0     0    12]
 [11675     0     0     0]
 [ 1839     0     0     0]
 [  638     0     0    24]]
              precision    recall  f1-score   support

           0       0.65      1.00      0.79     26759
           1       0.00      0.00      0.00     11675
           2       0.00      0.00      0.00      1839
         700       0.67      0.04      0.07       662

    accuracy                           0.65     40935
   macro avg       0.33      0.26      0.21     40935
weighted avg       0.44      0.65      0.52     40935



 - - - - - Terminado teste com 3 respostas - - - - - 




 - - - - - Começando teste com 4 respostas - - - - - 


[[26867     0     0     0    14]
 [11594     0     0     0     1]
 [ 1834     0     0     0     0]
 [  323     0     0     0     0]
 [  272     0     0     0    30]]
              precision    recall  f1-score   support

           0       0.66      1.00      0.79     26881
       

C:\Users\Matheus\miniconda\envs\BigQueryEnv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Matheus\miniconda\envs\BigQueryEnv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Matheus\miniconda\envs\BigQueryEnv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)